## Предобработка данных

Расшифровка названий признаков:
* car_description - марка, модель и поколение автомобиля
* year - год выпуска
* engine - объем двигателя в литрах
* power - мощность двигателя в лошадиных силах
* gas - типа используемого топлива
* transmission - тип коробки передач
* body - форма кузова
* drive - привод
* color - цвет кузова
* mileage - пробег
* price - цена

После того как отработал скрипт для парсинга web_scraper.py мы получили сырые данные в виде csv-файла auto_ru_df, но эти данные еще не готовы для визуализации и применения к ним алгоритмов машинного обучения. Чтобы с полученной информацией можно было работать - нам необходимо удалить лишние символы в колонках пробег и цена и преобразовать их к числовому типу. Так же нужно удалить дубликаты записей, проверить значения признаков в столбцах на адекватность и вытянуть максимум информации из колонки "car_decription",  где содержится краткое описание производителя, модели, поколения и тд. Приступим.

In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('precision', 2)

In [3]:
df = pd.read_csv('../data/auto_ru_df.csv', index_col=False,
                 names = ['car_description','year', 'engine', 'power', 'gas', 'transmission', \
                          'body', 'drive', 'color','mileage', 'price'])

In [4]:
df.head()

,car_description,year,engine,power,gas,transmission,body,drive,color,mileage,price
0,LADA () 2106,1990,1.3,64,Бензин,механика,седан,задний,бежевый,100 000 км,21 000 ₽
1,LADA () 2104 5-speed,1990,1.5,72,Бензин,механика,универсал 5 дв.,задний,красный,91 000 км,99 000 ₽
2,LADA () 2106,1990,1.3,64,Бензин,механика,седан,задний,бежевый,32 000 км,80 000 ₽
3,LADA () 2107,1990,1.3,64,Бензин,механика,седан,задний,красный,250 000 км,45 000 ₽
4,LADA () 2106,1990,1.3,64,Бензин,механика,седан,задний,голубой,68 000 км,28 000 ₽


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114106 entries, 0 to 114105
Data columns (total 11 columns):
car_description    114106 non-null object
year               114106 non-null int64
engine             114106 non-null float64
power              114106 non-null int64
gas                114106 non-null object
transmission       114106 non-null object
body               114105 non-null object
drive              114105 non-null object
color              114105 non-null object
mileage            114105 non-null object
price              114105 non-null object
dtypes: float64(1), int64(2), object(8)
memory usage: 9.6+ MB


Удаляем запись с пропущенными значениями

In [6]:
df.dropna(how = 'any', inplace = True)

Удаляем лишнее из колонок

In [7]:
df['car_description'] = df['car_description'].astype('str').str.replace(r'\(\)','')
df['gas'] = df['gas'].str.replace('\s+','')
df['price'] = df['price'].str.replace('\s+','').str.replace('₽', '')
df['mileage'] = df['mileage'].str.replace('\s+', '').str.replace('км', '')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 114105 entries, 0 to 114104
Data columns (total 11 columns):
car_description    114105 non-null object
year               114105 non-null int64
engine             114105 non-null float64
power              114105 non-null int64
gas                114105 non-null object
transmission       114105 non-null object
body               114105 non-null object
drive              114105 non-null object
color              114105 non-null object
mileage            114105 non-null object
price              114105 non-null object
dtypes: float64(1), int64(2), object(8)
memory usage: 10.4+ MB


Приводим пробег и цену к числовому типу

In [8]:
df['mileage'] = df['mileage'].astype('int32')
df['price'] = df['price'].astype('int32')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 114105 entries, 0 to 114104
Data columns (total 11 columns):
car_description    114105 non-null object
year               114105 non-null int64
engine             114105 non-null float64
power              114105 non-null int64
gas                114105 non-null object
transmission       114105 non-null object
body               114105 non-null object
drive              114105 non-null object
color              114105 non-null object
mileage            114105 non-null int32
price              114105 non-null int32
dtypes: float64(1), int32(2), int64(2), object(6)
memory usage: 9.6+ MB


In [9]:
df.describe()

,year,engine,power,mileage,price
count,114105.00,114105.00,114105.00,114105.00,1.14e+05
mean,2009.22,2.22,160.17,143259.07,7.52e+05
std,5.68,2.48,79.57,87023.87,9.38e+05
min,1990.00,0.70,29.00,1.00,1.50e+04
25%,2006.00,1.60,106.00,84000.00,2.80e+05
50%,2010.00,2.00,141.00,130000.00,5.15e+05
75%,2013.00,2.50,184.00,188000.00,8.90e+05
max,2018.00,170.00,717.00,1000000.00,8.89e+07


Максимальный объем двигателя в 170 литров выглядит подозрительно, вероятно, люди опечатывались или путали с другими единицами измерения.Подрежем на разумном значении.

In [10]:
def cutter(x):
    if x < 10:
        return x
    else:
        return x / 100
    
    
df['engine'] = df['engine'].apply(cutter)
df.describe()

,year,engine,power,mileage,price
count,114105.00,114105.00,114105.00,114105.00,1.14e+05
mean,2009.22,2.18,160.17,143259.07,7.52e+05
std,5.68,0.88,79.57,87023.87,9.38e+05
min,1990.00,0.67,29.00,1.00,1.50e+04
25%,2006.00,1.60,106.00,84000.00,2.80e+05
50%,2010.00,2.00,141.00,130000.00,5.15e+05
75%,2013.00,2.50,184.00,188000.00,8.90e+05
max,2018.00,8.40,717.00,1000000.00,8.89e+07


In [11]:
df.head()

,car_description,year,engine,power,gas,transmission,body,drive,color,mileage,price
0,LADA 2106,1990,1.3,64,Бензин,механика,седан,задний,бежевый,100000,21000
1,LADA 2104 5-speed,1990,1.5,72,Бензин,механика,универсал 5 дв.,задний,красный,91000,99000
2,LADA 2106,1990,1.3,64,Бензин,механика,седан,задний,бежевый,32000,80000
3,LADA 2107,1990,1.3,64,Бензин,механика,седан,задний,красный,250000,45000
4,LADA 2106,1990,1.3,64,Бензин,механика,седан,задний,голубой,68000,28000


In [12]:
df.duplicated().value_counts()

False    113154
True        951
dtype: int64

Удаляем одинаковые записи

In [13]:
df.drop_duplicates(inplace = True)
df.duplicated().value_counts()

False    113154
dtype: int64

In [14]:
df.shape

(113154, 11)

Удаляем вторую часть названия производителя "Land Rover", чтобы правильно извлечь названия модели

In [15]:
df['car_description'] = df['car_description'].astype('str').str.replace('Rover','')

Вытаскиваем полезную информацию из столбца "car_description" и удаляем его

In [16]:
manufacturer = df['car_description'].apply(lambda x : x.split()[0])
model = df['car_description'].apply(lambda x : x.split()[1])
restyling = df['car_description'].str.contains('Рестайлинг').astype('int32')
df.insert(0, 'manufacturer', manufacturer)
df.insert(1, 'model', model)
df.insert(2, 'restyling', restyling)
df.drop('car_description', axis = 1, inplace = True);

In [17]:
df['manufacturer'] = df['manufacturer'].str.replace('Land','Land Rover')

In [18]:
df.head()

,manufacturer,model,restyling,year,engine,power,gas,transmission,body,drive,color,mileage,price
0,LADA,2106,0,1990,1.3,64,Бензин,механика,седан,задний,бежевый,100000,21000
1,LADA,2104,0,1990,1.5,72,Бензин,механика,универсал 5 дв.,задний,красный,91000,99000
2,LADA,2106,0,1990,1.3,64,Бензин,механика,седан,задний,бежевый,32000,80000
3,LADA,2107,0,1990,1.3,64,Бензин,механика,седан,задний,красный,250000,45000
4,LADA,2106,0,1990,1.3,64,Бензин,механика,седан,задний,голубой,68000,28000


Сохрание очищенные и подготовленные данные в файл preprocessed_auto_ru_df.csv

In [ ]:
df.to_csv('../data/preprocessed_auto_ru_df.csv', index=False)